In [6]:
# !pip uninstall  -y matplotlib  # or conda
!pip install matplotlib==3.2.2

  Using cached matplotlib-3.2.2-cp38-cp38-macosx_11_0_arm64.whl


In [4]:
import pandas as pd
from backtesting  import Backtest, Strategy
from backtesting.lib import crossover
from kite_trade import *
import ta
import datetime

/Users/akash_verma/miniconda3/envs/epat/lib/python3.8/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [11]:
enctoken = "79mOAIk9qDmIgVdbiLEPoIzZSURm4Bmh+B5czd4f59wPoIiH4u9RFH1keBRsSWVrjrIq/FTyC3MFkY33mcbJek1NxZ5LdL8+x+K6yukKC1XU+dSdMIW4Tg=="
kite = KiteApp(enctoken=enctoken)


In [283]:
def get_diffrent_time_data(instrument_token,start_dt, end_dt,interval):
    data = pd.DataFrame()
    range_of_date = pd.date_range(start_dt,end_dt)

    index_val_1 = [i for i in pd.Series(range_of_date).index if i%2 == 0]
    index_val_2 = [i for i in pd.Series(range_of_date).index if i%2 != 0]
    x = pd.Series(range_of_date).iloc[index_val_1].reset_index(drop = True)
    y = pd.Series(range_of_date).iloc[index_val_2].reset_index(drop = True)
    df_1 = pd.concat([x,y],axis = 1).rename(columns = {0:"start_dt",1:"end_dt"})
    df_1.start_dt = pd.to_datetime(df_1.start_dt)
    df_1.end_dt = pd.to_datetime(df_1.end_dt)
    df_1.dropna(inplace = True)
    
    for r,c in df_1.iterrows():
#         print(c.start_dt,c.end_dt)
        temp = pd.DataFrame(kite.historical_data(instrument_token,c.start_dt,c.end_dt,interval))
        data = data.append(temp)
    return data

In [284]:
# instument in nse
inst = pd.DataFrame(kite.instruments("NSE"))
inst_filter = inst.query('(segment == "NSE") & (name != "")')

In [286]:
# inst_filter.instrument_token.unique()

In [287]:
class DayTrader(Strategy):
    
    def init(self):
        close = self.data.Close
        self.macd = self.I(ta.trend.macd,pd.Series(close))
        self.macd_signal = self.I(ta.trend.macd_signal,pd.Series(close))
        self.ema_100 = self.I(ta.trend.ema_indicator,pd.Series(close),window = 100)
    
    
    def next(self):
        price = self.data.Close
        
        if crossover(self.macd, self.macd_signal) and price > self.ema_100:
            sl = price * .97
            tp = price * 1.045
            self.buy(sl = sl , tp = tp)
            
        
    

In [290]:
for r,c in inst_filter.iterrows():
    instrument_token = c.instrument_token
    start_dt = datetime.datetime(2022,1,1)    # From last & days
    end_dt = datetime.datetime.now()
    interval = "15minute"
    
    data = get_diffrent_time_data(instrument_token,start_dt, end_dt,interval)
    data.date = pd.to_datetime(data.date,unit = "ms")
    data.set_index('date', inplace = True)
    data.columns = ['Open','High','Low',"Close","Volumne"]
    
    bt = Backtest(data,DayTrader, cash = 10000, commission=.0015)
    output = bt.run()
    print(f"{c.tradingsymbol} returns:{output['Return [%]']}")

20MICRONS returns:12.807312499999716
21STCENMGM returns:14.483366499999903
3IINFOLTD returns:-6.57234925000017


/var/folders/wy/0pkxqbpn0b15c5g3579k2g4w0000gn/T/ipykernel_2471/1913667369.py:12: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data,DayTrader, cash = 10000, commission=.0015)


3MINDIA returns:0.0
3PLAND returns:-80.56125050000024
4THDIM returns:84.90010249999992
5PAISA returns:-20.480223000000112



KeyboardInterrupt



In [277]:
bt.plot()

Row(id='5699', ...)

NameError: name 'data' is not defined